In [1]:
import pandas as pd

In [2]:
df_asd = pd.read_csv(r"C:\Vision\Project\thumos14-human-searches.csv")

In [3]:
df_asd.head(5)

,search-id,step-id,time,video-id,target-action
0,0,0,340.806,video_validation_0000413,JavelinThrow
1,0,1,333.616,video_validation_0000413,JavelinThrow
2,0,2,332.897,video_validation_0000413,JavelinThrow
3,0,3,331.459,video_validation_0000413,JavelinThrow
4,0,4,330.740,video_validation_0000413,JavelinThrow


In [4]:
len(df_asd['target-action'].unique())

20

In [5]:
actions_considered = df_asd['target-action'].unique()

In [6]:
videos_considered = df_asd['video-id'].unique()
#print (videos_considered)

In [7]:
import cv2
duration_dict = {}
for v in videos_considered:
    temp = cv2.VideoCapture("C:\Vision\Project\THUMOS\\" + v + ".mp4")
    temp.set(cv2.CAP_PROP_POS_AVI_RATIO,1)
    duration = temp.get(cv2.CAP_PROP_POS_MSEC)
    duration_dict[v] = duration/1000

In [8]:
#print (duration_dict)

In [9]:
df_asd[(df_asd['video-id'] == 'video_validation_0000789')]['search-id'].unique()

array([1423, 1548, 1572, 1583, 1674, 1692, 1725, 1730, 1738, 1754, 1791],
      dtype=int64)

# Random Baseline

In [11]:
df_thumos_test = pd.read_csv(r"C:\Vision\Project\thumos14-train.csv", names  = ['video_id', 'starting_time', 'ending_time', 'action_id'])

In [12]:
df_thumos_test.head(5)

,video_id,starting_time,ending_time,action_id
0,video_validation_0000266,72.8,76.4,7
1,video_validation_0000681,44.0,50.9,7
2,video_validation_0000682,1.5,5.4,7
3,video_validation_0000682,79.3,83.9,7
4,video_validation_0000683,0.3,5.5,7


In [13]:
videos = df_thumos_test['video_id'].unique()
print (len(videos))
print (len(videos_considered))

act_dict = {7 : 'BaseballPitch',
9 : 'BasketballDunk',
12: 'Billiards',
21: 'CleanAndJerk',
22: 'CliffDiving',
23: 'CricketBowling',
24: 'CricketShot',
26: 'Diving',
31: 'FrisbeeCatch',
33: 'GolfSwing',
36: 'HammerThrow',
40: 'HighJump',
45: 'JavelinThrow',
51: 'LongJump',
68: 'PoleVault',
79: 'Shotput',
85: 'SoccerPenalty',
92: 'TennisSwing',
93: 'ThrowDiscus',
97: 'VolleyballSpiking'}


200
195


In [18]:
import random

def find_action(data1, data2, video_id) :    
    
    search_num = 0;
    min_local = 0
    max_local = duration_dict[video_id]
    for i in range(500) :
        guess_time = random.uniform(min_local, max_local)
        flag = 0
#         for j in range(len(data1)):
#             if guess_time > data1[j] and guess_time < data2[j]:
#                 flag = 1
#                 break;
        if guess_time > data1[0] and guess_time < data2[0]:
            flag = 1
            break;
        
        search_num = search_num + 1
        if flag == 1:
            break;
    
    return search_num

In [19]:
epochs = 10
# For particular video
action_spot = {}
count_action = {}
for video in videos :
    if video in videos_considered:
    
        df_thumos_video = df_thumos_test[(df_thumos_test['video_id'] == video)]

        avg = 0;
        count = 0;

        for epoch in range(epochs) :

    #         For particular action
            video_actions = df_thumos_video['action_id'].unique()
            #print('Video', video)

            for action in video_actions :            
                temp = df_thumos_video[(df_thumos_video['action_id'] == action)]

                as_metric = 500
                while (as_metric == 500) :
                    as_metric = find_action(temp['starting_time'].tolist(), temp['ending_time'].tolist(), video)

                if action in action_spot:
                    action_spot[action] = action_spot[action] + as_metric
                    count_action[action] = count_action[action] + 1
                else:
                    action_spot[action] = as_metric
                    count_action[action] = 1

                #print('Action : %s , Spotted in : %i' % (act_dict[action], as_metric))
                avg = avg + as_metric
                count = count + 1
    


In [20]:
results = {}
for key, value in action_spot.items():
    results[key] = action_spot[key]/count_action[key]
        

In [21]:
print (results)

{7: 27.054545454545455, 31: 43.1, 9: 103.86666666666666, 12: 79.11, 21: 12.822222222222223, 22: 44.53, 26: 61.863157894736844, 23: 78.28823529411764, 24: 92.59375, 33: 35.6, 92: 78.56, 36: 52.41, 40: 83.1, 45: 84.56, 51: 41.3, 68: 42.85, 79: 29.24, 93: 16.463636363636365, 85: 29.68, 97: 59.266666666666666}
